### Advanced Python Constructs

**Iteratro**

An iterator is an object adhering to the `iterator protocol` — basically this means that it has a next `<iterator.next>` method, which, when called, returns the next item in the sequence, and when there’s nothing to return, raises the `StopIteration` `<exceptions.StopIteration>` exception.

alling the `__iter__ <object.__iter__>` method on a container to create an iterator object is the most straightforward way to get hold of an iterator. The `iter` function does that for us.

In [1]:
nums = [1, 2, 3]      # note that ... varies: these are different objects
iter(nums)

In [2]:
nums.__iter__()

In [3]:
nums.__reversed__()

In [4]:
it = iter(nums)
next(it)

1

In [ ]:
next(it)
next(it)
next(it) # raises StopIteration

StopIteration: 

When used in a loop, StopIteration `<exceptions.StopIteration>` is swallowed and causes the loop to finish. But with explicit invocation, we can see that once the iterator is exhausted, accessing it raises an exception.

Using the `for..in` loop also uses the `__iter__` method. This allows us to transparently start the iteration over a sequence. But if we already have the iterator, we want to be able to use it in an `for` loop in the same way. In order to achieve this, iterators in addition to `next` are also required to have a method called `__iter__` which returns the iterator (`self`).

Support for iteration is pervasive in Python: all sequences and unordered containers in the standard library allow this. The concept is also stretched to other things: e.g. `file` objects support iteration over lines.

> ```python 
> with open("/etc/fstab") as f:
>      assert f is f.__iter__()
> ```
The `file` is an iterator itself and its `__iter__` method doesn’t create a separate object: only a single thread of sequential access is allowed.

---
### **Generator expressions**
A second way in which iterator objects are created is through **generator expressions**, the basis for **list comprehensions**. To increase clarity, a generator expression must always be enclosed in parentheses or an expression. If round parentheses are used, then a generator iterator is created. If rectangular parentheses are used, the process is short-circuited and we get a `list`.

In [6]:
(i for i in nums)

<generator object <genexpr> at 0x7f11a123cc40>

In [7]:
[i for i in nums]

[1, 2, 3]

In [8]:
list(i for i in nums)

[1, 2, 3]

The list comprehension syntax also extends to dictionary and `set` comprehensions. A set is created when the generator expression is enclosed in curly braces. 

A `dict` is created when the generator expression contains “pairs” of the form `key:value`:

In [9]:
{i for i in range(3)}

{0, 1, 2}

In [10]:
{i:i**2 for i in range(3)}

{0: 0, 1: 1, 2: 4}

---
### **Generators**
A third way to create iterator objects is to call a **generator** function. A **generator** is a function containing the keyword `yield`. It must be noted that the mere presence of this keyword completely changes the nature of the function- this `yield` statement doesn’t have to be invoked, or even reachable, but causes the function to be marked as a generator. When a normal function is called, the instructions contained in the body start to be executed. 

When a generator is called, the execution stops before the first instruction in the body. An invocation of a generator function creates a generator object, adhering to the iterator protocol. As with normal function invocations, concurrent and recursive invocations are allowed.

When `next` is called, the function is executed until the first `yield`. Each encountered `yield` statement gives a value becomes the return value of `next`. After executing the `yield` statement, the execution of this function is suspended.

In [11]:
def f():
    yield 1
    yield 2

f()

<generator object f at 0x7f11a0dc8510>

In [12]:
gen = f()
next(gen)

1

In [13]:
next(gen)

2

In [14]:
next(gen)

StopIteration: 

In [ ]:
#Let’s go over the life of the single invocation of the generator function.

def f():
    print("-- start --")
    yield 3
    print("-- finish --")
    yield 4

gen = f()
next(gen)

-- start --
-- finish --


In [16]:
next(gen)

-- finish --


4

In [17]:
next(gen)

StopIteration: 

---
### **Bidirectional communication**
Each `yield` statement causes a value to be passed to the caller. This is the reason for the introduction of generators by PEP 255. But communication in the reverse direction is also useful. One obvious way would be some external state, either a global variable or a shared mutable object. Direct communication is possible thanks to PEP 342. It is achieved by turning the previously boring `yield` statement into an expression. When the generator resumes execution after a `yield` statement, the caller can call a method on the generator object to either pass a value into the generator, which then is returned by the yield statement, or a different method to inject an exception into the generator.

The first of the new methods is `send(value) <generator.send>`, which is similar to `next() <generator.next>`, but passes `value` into the generator to be used for the value of the `yield` expression. In fact, `g.next()` and `g.send(None)` are equivalent.

The second of the new methods is `throw(type, value=None, traceback=None) <generator.throw>` which is equivalent to-
```python
raise type, value, traceback
```
at the point of the yield statement.

Unlike `raise` (which immediately raises an exception from the current execution point), `throw() first resumes the generator, and only then raises the exception. The word throw was picked because it is suggestive of putting the exception in another location, and is associated with exceptions in other languages.

What happens when an exception is raised inside the generator? It can be either raised explicitly or when executing some statements or it can be injected at the point of a `yield` statement by means of the `throw()` method. In either case, such an exception propagates in the standard manner: it can be intercepted by an `except` or `finally` clause, or otherwise it causes the execution of the generator function to be aborted and propagates in the caller.

For completeness’ sake, it’s worth mentioning that generator iterators also have a `close() <generator.close>` method, which can be used to force a generator that would otherwise be able to provide more values to finish immediately. It allows the generator `__del__ <object.__del__>` method to destroy objects holding the state of generator. Let’s define a generator which just prints what is passed in through send and throw.

In [19]:
import itertools

def g():
    print('--start--')
    for i in itertools.count():
        print('--yielding %i--' % i)
        try:
            ans = yield i
        except GeneratorExit:
            print('--closing--')
            raise
        except Exception as e:
            print('--yield raised %r--' % e)
        else:
            print('--yield returned %s--' % ans)

In [20]:
it = g()
next(it)

--start--
--yielding 0--


0

In [21]:
it.send(11)

--yield returned 11--
--yielding 1--


1

In [22]:
it.throw(IndexError)

--yield raised IndexError()--
--yielding 2--


2

In [23]:
it.close()

--closing--


---
### **Chaining generators**
> Note
> 
> This is a preview of PEP 380 (not yet implemented, but accepted for Python 3.3).

Let’s say we are writing a generator and we want to yield a number of values generated by a second generator, a subgenerator. If yielding of values is the only concern, this can be performed without much difficulty using a loop such as

```python
subgen = some_other_generator()
for v in subgen:
    yield v
```
However, if the subgenerator is to interact properly with the caller in the case of calls to `send()`, `throw()` and `close()`, things become considerably more difficult. The `yield` statement has to be guarded by a `try..except..finally` structure similar to the one defined in the previous section to “debug” the generator function. Such code is provided in PEP 380#id13, here it suffices to say that new syntax to properly yield from a subgenerator is being introduced in Python 3.3:
```python
yield from some_other_generator()
```
This behaves like the explicit loop above, repeatedly yielding values from `some_other_generator` until it is exhausted, but also forwards `send`, `throw` and `close` to the subgenerator.

In [ ]:
def simple_decorator(function):
  print("doing decoration")
  return function

@simple_decorator
def function():
  print("inside function")

doing decoration


```python
@simple_decorator
def function():
    ...
```
is executed as :

```python
function = simple_decorator(function)
```

In [25]:
function()

inside function


In [ ]:
def decorator_with_arguments(arg):
  print("defining the decorator")
  def _decorator(function):
      # in this inner function, arg is available too
      print("doing decoration, %r" % arg)
      return function
  return _decorator

@decorator_with_arguments("abc")
def function():
  print("inside function")

defining the decorator
doing decoration, 'abc'


In [27]:
function()

inside function


The two trivial decorators above fall into the category of decorators which return the original function. If they were to return a new function, an extra level of nestedness would be required. In the worst case, three levels of nested functions.

In [28]:
def replacing_decorator_with_args(arg):
  print("defining the decorator")
  def _decorator(function):
      # in this inner function, arg is available too
      print("doing decoration, %r" % arg)
      def _wrapper(*args, **kwargs):
          print("inside wrapper, %r %r" % (args, kwargs))
          return function(*args, **kwargs)
      return _wrapper
  return _decorator
@replacing_decorator_with_args("abc")
def function(*args, **kwargs):
    print("inside function, %r %r" % (args, kwargs))
    return 14

defining the decorator
doing decoration, 'abc'


In [29]:
function(11, 12)

inside wrapper, (11, 12) {}
inside function, (11, 12) {}


14